In [1]:
import numpy as np
import pandas as pd
import anndata

from scipy.stats import mode, pearsonr
from sklearn.metrics import pairwise_distances, adjusted_mutual_info_score, silhouette_score
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import HDBSCAN
import warnings

### Pachter metrics


In [2]:
def centroid_distances(data,labels,return_flat=True,metric='euclidean'):
    
    clusters = np.unique(labels)
    centroids=[]
    for i,cl in enumerate(clusters):
        
        data_cluster = data[labels==cl,:]
        centroids.append(np.mean(data_cluster,axis=0))
        
    centroids = np.vstack(centroids)
    distances = pairwise_distances(centroids,centroids,metric=metric)
    
    if return_flat:
        return distances.flatten()
    else:
        return distances

def intra_cluster_avg_distance(data,labels,metric='euclidean'):
    
    clusters = np.unique(labels)
    avg_distances = []
    for i,cl in enumerate(clusters):
        data_cluster = data[labels==cl,:]
        distances = pairwise_distances(data_cluster,data_cluster,metric=metric)
        np.fill_diagonal(distances, np.nan)
        avg_distances.append(np.nanmean(distances))
        
    return avg_distances

### kNN Metrics

In [3]:
def knn_acc_loocv(X, y, n_neighbors=10):
    neigh = NearestNeighbors(n_neighbors=n_neighbors).fit(X)
    knn = neigh.kneighbors(return_distance=False)
    yhat = mode(y[knn], axis=1).mode.flatten()
    return np.mean(yhat == y)

def knn_recall(x_hd, x_ld, n_neighbors=10):
    
    assert len(x_hd)==len(x_ld)
    
    neigh_hd = NearestNeighbors(n_neighbors=n_neighbors).fit(x_hd)
    neigh_ld = NearestNeighbors(n_neighbors=n_neighbors).fit(x_ld)
    knn_hd = neigh_hd.kneighbors(return_distance=False)
    knn_ld = neigh_ld.kneighbors(return_distance=False)
    
    n_recalled_neighb=[]
    for i in range(len(knn_ld)):
        n_recalled_neighb.append(len(np.intersect1d(knn_ld[i,:],knn_hd[i,:])))
    recall=np.mean(np.array(n_recalled_neighb)/n_neighbors)
    
    return recall

### Compute kNN metrics, silhouette score and `Chari & Pachter 2023` metrics

In [4]:
def compute_metrics(ad, embedding_type_input=['pca2','elephant','tsne','umap',]):
    
    x_hd = ad.X # C&P always evaluated on the un-scaled space, even though they used the scaled space to compute their embeddings
    labels = ad.obs['clusterlabels']
    dataset = ad.uns['dataset']
    
    #compute Pachter distance metrics for high dimensional space
    centroid_dist_hd = centroid_distances(x_hd,labels)
    intra_dist_hd = intra_cluster_avg_distance(x_hd,labels)
    centroid_dist_hd_l1 = centroid_distances(x_hd,labels,metric='l1')
    intra_dist_hd_l1 = intra_cluster_avg_distance(x_hd,labels,metric='l1')

    metric_values = []
    metric_types = []
    embedding_types = []
    seeds = []
    scaled = []
    datasets=[]
    
    #compute for scaled and unscaled data for main datasets
    if dataset in ['exut','merfish','smartseq']:
        use_scaled_modes = [False, True]
        print('running on scaled and unscaled data for', dataset)
    #only use unscaled for additional datasets
    else:
        use_scaled_modes = [False]
        print('running only on unscaled data for', dataset)
        
    for use_scaled in use_scaled_modes:
        
        if use_scaled:
            scaled_str = '_scaled'
        else:
            scaled_str = ''
    
        for embedding_type in embedding_type_input:

            for seed in ad.uns['seeds']:
                print(dataset,scaled_str[1:],embedding_type,'run',seed)
                x_ld = ad.obsm[f'x{scaled_str}_{embedding_type}_seed_{seed}']

                ### pachter metrics (L2)
                centroid_dist_ld = centroid_distances(x_ld,labels)
                intra_dist_ld = intra_cluster_avg_distance(x_ld,labels)            
                corr_centroid_dist = pearsonr(centroid_dist_ld,centroid_dist_hd)[0]
                corr_intra_dist = pearsonr(intra_dist_ld,intra_dist_hd)[0]

                ### pachter L1 control
                centroid_dist_ld_l1 = centroid_distances(x_ld,labels,metric='l1')
                intra_dist_ld_l1 = intra_cluster_avg_distance(x_ld,labels,metric='l1')
                corr_centroid_dist_l1 = pearsonr(centroid_dist_ld_l1,centroid_dist_hd_l1)[0]
                corr_intra_dist_l1 = pearsonr(intra_dist_ld_l1,intra_dist_hd_l1)[0]

                ### kNN metrics
                # return x_ld,labels
                accurray = knn_acc_loocv(x_ld,labels.values)
                recall = knn_recall(x_hd,x_ld)
                
                ### silhouette score
                silhouette = silhouette_score(X=x_ld,labels=labels,random_state=0)

                ### save to arrays
                metric_values += [corr_centroid_dist,corr_intra_dist,
                                  corr_centroid_dist_l1,corr_intra_dist_l1,
                                  accurray,recall,
                                  silhouette]
                metric_types += ['inter-centroid distance corr','intra-cluster distance corr',
                                 'inter-centroid L1 distance corr','intra-cluster L1 distance corr',
                                 'kNN accuracy','kNN recall',
                                 'silhouette_score']
                embedding_types += [embedding_type]*7
                seeds += [seed]*7
                datasets += [dataset]*7
                scaled += [use_scaled]*7
    
    ### make dataframe
    metrics_df = pd.DataFrame(dict(metric_value=metric_values,
                                  metric=metric_types,
                                  embedding_type=embedding_types,
                                  seed=seeds,
                                  dataset=datasets,
                                  scaled=scaled))

    ad.uns['knn_accuracy_hd'] = knn_acc_loocv(x_hd,labels.values)    
    ad.uns['silhouette_hd'] = silhouette_score(x_hd,labels=labels.values,metric='euclidean')
    
    ad.uns['metrics_df'] = metrics_df

In [5]:
adata_exut=anndata.read_h5ad("../results/embeddings/exut_adata_all_embeddings.h5ad")
adata_merfish=anndata.read_h5ad("../results/embeddings/merfish_adata_all_embeddings.h5ad")
adata_smartseq=anndata.read_h5ad("../results/embeddings/smartseq_adata_all_embeddings.h5ad")
adata_mnist=anndata.read_h5ad("../results/embeddings/mnist_adata_all_embeddings.h5ad")
adata_sim = anndata.read_h5ad('../results/embeddings/exut-sim-theta-10-real-seqdepths_adata_all_embeddings.h5ad')
datasets = [adata_exut]#[adata_exut,adata_merfish,adata_smartseq,adata_mnist,adata_sim]

In [6]:
%%time
[compute_metrics(ad) for ad in datasets]

running on scaled and unscaled data for exut
exut  pca2 run 0
exut  pca2 run 1
exut  pca2 run 2
exut  pca2 run 3
exut  pca2 run 4
exut  elephant run 0
exut  elephant run 1
exut  elephant run 2
exut  elephant run 3
exut  elephant run 4
exut  tsne run 0
exut  tsne run 1
exut  tsne run 2
exut  tsne run 3
exut  tsne run 4
exut  umap run 0
exut  umap run 1
exut  umap run 2
exut  umap run 3
exut  umap run 4
exut scaled pca2 run 0
exut scaled pca2 run 1
exut scaled pca2 run 2
exut scaled pca2 run 3
exut scaled pca2 run 4
exut scaled elephant run 0
exut scaled elephant run 1
exut scaled elephant run 2
exut scaled elephant run 3
exut scaled elephant run 4
exut scaled tsne run 0
exut scaled tsne run 1
exut scaled tsne run 2
exut scaled tsne run 3
exut scaled tsne run 4
exut scaled umap run 0
exut scaled umap run 1
exut scaled umap run 2
exut scaled umap run 3
exut scaled umap run 4
CPU times: user 9min 32s, sys: 4min 7s, total: 13min 39s
Wall time: 1min 17s


[None]

### AMI metric

#### HDBSCAN Clustering in 2D

In [7]:
manycolors = ["#FFFF00", "#1CE6FF", "#FF34FF", "#FF4A46", "#008941", "#006FA6", "#A30059",
        "#FFDBE5", "#7A4900", "#0000A6", "#63FFAC", "#B79762", "#004D43", "#8FB0FF", "#997D87",
        "#5A0007", "#809693", "#FEFFE6", "#1B4400", "#4FC601", "#3B5DFF", "#4A3B53", "#FF2F80",
        "#61615A", "#BA0900", "#6B7900", "#00C2A0", "#FFAA92", "#FF90C9", "#B903AA", "#D16100",
        "#DDEFFF", "#000035", "#7B4F4B", "#A1C299", "#300018", "#0AA6D8", "#013349", "#00846F",
        "#372101", "#FFB500", "#C2FFED", "#A079BF", "#CC0744", "#C0B9B2", "#C2FF99", "#001E09",
        "#00489C", "#6F0062", "#0CBD66", "#EEC3FF", "#456D75", "#B77B68", "#7A87A1", "#788D66",
        "#885578", "#FAD09F", "#FF8A9A", "#D157A0", "#BEC459", "#456648", "#0086ED", "#886F4C",
        "#34362D", "#B4A8BD", "#00A6AA", "#452C2C", "#636375", "#A3C8C9", "#FF913F", "#938A81",
        "#575329", "#00FECF", "#B05B6F", "#8CD0FF", "#3B9700", "#04F757", "#C8A1A1", "#1E6E00",
        "#7900D7", "#A77500", "#6367A9", "#A05837", "#6B002C", "#772600", "#D790FF", "#9B9700",
        "#549E79", "#FFF69F", "#201625", "#72418F", "#BC23FF", "#99ADC0", "#3A2465", "#922329",
        "#5B4534", "#FDE8DC", "#404E55", "#0089A3", "#CB7E98", "#A4E804", "#324E72", "#6A3A4C",
        "#83AB58", "#001C1E", "#D1F7CE", "#004B28", "#C8D0F6", "#A3A489", "#806C66", "#222800",
        "#BF5650", "#E83000", "#66796D", "#DA007C", "#FF1A59", "#8ADBB4", "#1E0200", "#5B4E51",
        "#C895C5", "#320033", "#FF6832", "#66E1D3", "#CFCDAC", "#D0AC94", "#7ED379", "#012C58"]

excluded_color_ids = [0,7,17,31,41,45,50,88,96,105] #excluded for low contrast on white
manycolors = [c for i,c in enumerate(manycolors) if not i in excluded_color_ids]
colors_tab10 = ['tab:blue','tab:orange','tab:green','tab:red','tab:purple','tab:brown','tab:pink','tab:gray','tab:olive','tab:cyan']

def generate_hdb_colors(hdb_labels):
    
    n_hdb_clusters = sum(np.unique(hdb_labels)>=0)
    if n_hdb_clusters <= 10:
        colors_for_cycle = colors_tab10 
    else:
        colors_for_cycle = manycolors    
    colors_cycle = np.array(colors_for_cycle * int(np.ceil((max(hdb_labels)+1)/len(colors_for_cycle))))
    colors_hdb = colors_cycle[hdb_labels]
    
    background_idx = hdb_labels==-1
    colors_hdb[background_idx]='k'
    
    return colors_hdb

In [8]:
def compute_hdb_scan(ad,use_scaled=False,min_samples=5,min_cluster_size=5,seed_to_use=0,embedding_type='tsne',allow_single_cluster=True):
    
    hdb = HDBSCAN(min_samples=min_samples,
                 min_cluster_size=min_cluster_size,
                 allow_single_cluster=allow_single_cluster)

    if use_scaled:
        scaled_str = '_scaled'
    else:
        scaled_str = ''
    
    x=ad.obsm[f'x{scaled_str}_{embedding_type}_seed_{seed_to_use}']
    
    id_str_core = f'x{scaled_str}_{embedding_type}_seed_{seed_to_use}_min_samples_{min_samples}_min_cluster_size_{min_cluster_size}_allow_single_cluster_{allow_single_cluster}'
    id_str_labels = f'hdb_labels_{id_str_core}'
    id_str_color = f'hdb_colors_{id_str_core}'
    
    if id_str_labels in ad.obs.keys():
        print(f'skipping {id_str_core}')
        return
    
    else:
        hdb.fit(x)
        hdb_labels = hdb.labels_
        ad.obs[id_str_labels] = hdb_labels
        ad.obs[id_str_color] = generate_hdb_colors(hdb_labels)

        np.save(f'../results/clusterings/npy/hdb_{id_str_core}.npy',hdb)

        return

### Compute 2D clustering

In [10]:
%%time
min_samples_all = np.array([5,10,15,20,30,40,50,75,100])
embedding_types = ['tsne','umap','pca2','elephant']

for ad in datasets:
    
    dataset=ad.uns['dataset']
    
    #compute for scaled and unscaled data for main datasets
    if dataset in ['exut','merfish','smartseq']:
        use_scaled_modes = [False, True]
        print('running on scaled and unscaled data for', dataset)
    #only use unscaled for additional datasets
    else:
        use_scaled_modes = [False]
        print('running only on unscaled data for', dataset)
        
    for use_scaled in use_scaled_modes:
        for embedding_type in embedding_types:
            for seed_to_use in np.arange(5):
                for min_samples in min_samples_all:
                    print(dataset,'scaled:',use_scaled,embedding_type,'run',seed_to_use,'hdb-scan-param',min_samples)
                    
                    with warnings.catch_warnings():
                        warnings.simplefilter("ignore", category=pd.errors.PerformanceWarning)
                        compute_hdb_scan(ad,
                                         use_scaled=use_scaled,
                                         min_samples=min_samples,
                                         min_cluster_size=min_samples,
                                         allow_single_cluster=False,
                                         embedding_type=embedding_type,
                                         seed_to_use=seed_to_use)

running on scaled and unscaled data for exut
exut scaled: False tsne run 0 hdb-scan-param 5
exut scaled: False tsne run 0 hdb-scan-param 10
exut scaled: False tsne run 0 hdb-scan-param 15
exut scaled: False tsne run 0 hdb-scan-param 20
exut scaled: False tsne run 0 hdb-scan-param 30
exut scaled: False tsne run 0 hdb-scan-param 40
exut scaled: False tsne run 0 hdb-scan-param 50
exut scaled: False tsne run 0 hdb-scan-param 75
exut scaled: False tsne run 0 hdb-scan-param 100
exut scaled: False tsne run 1 hdb-scan-param 5
exut scaled: False tsne run 1 hdb-scan-param 10
exut scaled: False tsne run 1 hdb-scan-param 15
exut scaled: False tsne run 1 hdb-scan-param 20
exut scaled: False tsne run 1 hdb-scan-param 30
exut scaled: False tsne run 1 hdb-scan-param 40
exut scaled: False tsne run 1 hdb-scan-param 50
exut scaled: False tsne run 1 hdb-scan-param 75
exut scaled: False tsne run 1 hdb-scan-param 100
exut scaled: False tsne run 2 hdb-scan-param 5
exut scaled: False tsne run 2 hdb-scan-param

In [11]:
def compute_ami(ad,
                use_scaled=False,
                embedding_type='tsne',
                min_samples = 30,
                min_cluster_size = 30,
                allow_single_cluster = False,
                seed_to_use = 0,
                k=1):

    if use_scaled:
        scaled_str = '_scaled'
    else:
        scaled_str = ''
    
    x=ad.obsm[f'x{scaled_str}_{embedding_type}_seed_{seed_to_use}']

    id_str_core = f'x{scaled_str}_{embedding_type}_seed_{seed_to_use}_min_samples_{min_samples}_min_cluster_size_{min_cluster_size}_allow_single_cluster_{allow_single_cluster}'
    id_str_labels = f'hdb_labels_{id_str_core}'
    id_str_colors = f'hdb_colors_{id_str_core}'   


    labels_hdb = ad.obs[id_str_labels]
    colors_hdb = ad.obs[id_str_colors]

    background_idx = labels_hdb==-1
    if sum(background_idx)>0:
        x_no_noise = x[~background_idx,:]
        labels_no_noise = labels_hdb[~background_idx].values
        x_noise = x[background_idx,:]

        neighbors = NearestNeighbors(n_neighbors=k).fit(x_no_noise)
        knn = neighbors.kneighbors(x_noise,return_distance=False)
        closest_label_noisepoints = mode(labels_no_noise[knn], axis=1).mode.flatten()

        labels_hdb_clean = labels_hdb.copy()
        labels_hdb_clean[background_idx]=closest_label_noisepoints
        colors_hdb_clean = generate_hdb_colors(labels_hdb_clean)

    else:
        labels_hdb_clean = labels_hdb.copy()
        colors_hdb_clean = colors_hdb.copy()

    n_cluster_hdb = len(np.unique(labels_hdb_clean))
    ami = adjusted_mutual_info_score(ad.obs['clusterlabels'],labels_hdb_clean)
    
    return ami,n_cluster_hdb

In [12]:
%%time
for ad in datasets:
    ami_dfs = []
    
    dataset=ad.uns['dataset']

    #compute for scaled and unscaled data for main datasets
    if dataset in ['exut','merfish','smartseq']:
        use_scaled_modes = [False, True]
        print('running on scaled and unscaled data for', dataset)
    #only use unscaled for additional datasets
    else:
        use_scaled_modes = [False]
        print('running only on unscaled data for', dataset)
        
    for use_scaled in use_scaled_modes:
        for embedding_type in ['pca2','elephant','tsne','umap']:
            for seed_to_use in np.arange(5):
                print(dataset,'scaled:',use_scaled,embedding_type,'run',seed_to_use)
            
                ### AMI
                for hdb_scan_param in [5,10,15, 20,30,40, 50,75,100]:

                    ami,n_cluster_hdb = compute_ami(ad=ad,
                                                        min_samples=hdb_scan_param,
                                                        min_cluster_size=hdb_scan_param,
                                                        embedding_type=embedding_type,
                                                        seed_to_use=seed_to_use,
                                                        use_scaled=use_scaled)

                    ami_dfs += [pd.DataFrame(data=dict(metric_value=[ami],
                                          metric=['ami'],
                                          embedding_type=[embedding_type],
                                          seed=[seed_to_use],
                                          dataset=[ad.uns['dataset']],
                                          hdb_scan_param=hdb_scan_param,
                                          n_hdb_scan_clusters=n_cluster_hdb,
                                          scaled=use_scaled))]

    ad.uns['ami_df'] = pd.concat(ami_dfs)

    dataset = ad.uns['dataset']    
    ad.write_h5ad(f'../results/evaluations/{dataset}_adata_evaluated_embeddings.h5ad')

running on scaled and unscaled data for exut
exut scaled: False pca2 run 0
exut scaled: False pca2 run 1
exut scaled: False pca2 run 2
exut scaled: False pca2 run 3
exut scaled: False pca2 run 4
exut scaled: False elephant run 0
exut scaled: False elephant run 1
exut scaled: False elephant run 2
exut scaled: False elephant run 3
exut scaled: False elephant run 4
exut scaled: False tsne run 0
exut scaled: False tsne run 1
exut scaled: False tsne run 2
exut scaled: False tsne run 3
exut scaled: False tsne run 4
exut scaled: False umap run 0
exut scaled: False umap run 1
exut scaled: False umap run 2
exut scaled: False umap run 3
exut scaled: False umap run 4
exut scaled: True pca2 run 0
exut scaled: True pca2 run 1
exut scaled: True pca2 run 2
exut scaled: True pca2 run 3
exut scaled: True pca2 run 4
exut scaled: True elephant run 0
exut scaled: True elephant run 1
exut scaled: True elephant run 2
exut scaled: True elephant run 3
exut scaled: True elephant run 4
exut scaled: True tsne ru

### Package versions

In [13]:
np.__version__

'1.24.3'

In [14]:
pd.__version__

'2.2.1'

In [15]:
anndata.__version__

'0.10.3'

In [16]:
import scipy; scipy.__version__

'1.11.1'

In [17]:
import sklearn; sklearn.__version__

'1.3.0'